In [236]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [237]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import graycomatrix, graycoprops
import matplotlib.pyplot as plt

In [238]:
dataset_path = '/content/drive/MyDrive/Dataset_Daun'
paths = {
    "pepaya": os.path.join(dataset_path, "Dataset_Daun_Pepaya"),
    "pandan": os.path.join(dataset_path, "Dataset_Daun_Pandan")
}

In [239]:
def extract_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (200, 200))
    glcm = graycomatrix(resized, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    return [graycoprops(glcm, p)[0, 0] for p in props]

In [240]:
X, y = [], []

for label, folder in paths.items():
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        img = cv2.imread(file_path)
        if img is None:
            print(f"Gagal membaca: {file_path}")
            continue
        features = extract_glcm_features(img)
        X.append(features)
        y.append(label)

In [241]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [242]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[1 1]
 [1 1]]

Classification Report:
               precision    recall  f1-score   support

      pandan       0.50      0.50      0.50         2
      pepaya       0.50      0.50      0.50         2

    accuracy                           0.50         4
   macro avg       0.50      0.50      0.50         4
weighted avg       0.50      0.50      0.50         4



In [243]:
plt.figure(figsize=(6, 4))
plt.title("Perbandingan Prediksi vs Asli")
plt.plot(y_test, 'bo-', label='Asli')
plt.plot(y_pred, 'rx--', label='Prediksi')
plt.legend()
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [244]:
import joblib

model_path = '/content/drive/MyDrive/Dataset_Daun/model_knn.pkl'
joblib.dump(model, model_path)
print(f"✅ Model disimpan di: {model_path}")

✅ Model disimpan di: /content/drive/MyDrive/Dataset_Daun/model_knn.pkl


In [248]:
from google.colab import files
uploaded = files.upload()

Saving 8.jpg to 8.jpg


In [249]:
test_image_path = '8.jpg'

In [250]:
test_img = cv2.imread(test_image_path)
if test_img is None:
    print("Gambar tidak ditemukan!")
else:
    features = extract_glcm_features(test_img)
    prediction = model.predict([features])[0]
    print(f"🔍 Hasil prediksi: {prediction}")

🔍 Hasil prediksi: pandan


In [ ]:
  import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import graycomatrix, graycoprops

In [ ]:
def extract_glcm_features(image_gray):
    glcm = graycomatrix(image_gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=256, symmetric=True, normed=True)

    contrast = graycoprops(glcm, 'contrast').mean()
    dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    energy = graycoprops(glcm, 'energy').mean()
    correlation = graycoprops(glcm, 'correlation').mean()
    ASM = graycoprops(glcm, 'ASM').mean()

    return [contrast, dissimilarity, homogeneity, energy, correlation, ASM]

In [ ]:
dataset_path = '/content/drive/MyDrive/Dataset_Daun'
labels = []
features = []

In [ ]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue
    for file_name in os.listdir(class_folder):
        file_path = os.path.join(class_folder, file_name)
        image = cv2.imread(file_path)
        if image is None:
            continue
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_gray = cv2.resize(image_gray, (128, 128))  # Normalisasi ukuran
        feature = extract_glcm_features(image_gray)
        features.append(feature)
        labels.append(class_name)

In [ ]:
X = np.array(features)
y = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.25

Laporan Klasifikasi:
                     precision    recall  f1-score   support

Dataset_Daun_Pandan       0.33      0.50      0.40         2
Dataset_Daun_Pepaya       0.00      0.00      0.00         2

           accuracy                           0.25         4
          macro avg       0.17      0.25      0.20         4
       weighted avg       0.17      0.25      0.20         4



In [ ]:
mean_df = df.groupby('Kelas').mean().reset_index()
print(mean_df)

    Kelas    Contrast  Correlation    Energy  Homogeneity
0  Pandan  378.373541     0.911975  0.042410     0.224793
1  Pepaya  417.721268     0.936650  0.129988     0.327452
